In [1]:
import tensorflow as tf
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import sys
import importlib
from pathlib import Path

MY_SRC = '../src/'
if MY_SRC not in sys.path:
    sys.path.append(MY_SRC)

import text_model
import data

In [ ]:
# Sanity check on the data

g = tf.Graph()
with g.as_default():
    dataset = data.make_dataset('/home/derick/Downloads/testing_data.tfrecord')
    iterator = tf.data.Dataset.make_one_shot_iterator(dataset)
    with tf.Session() as sess:
        batch = sess.run(iterator.get_next())

print("".join(chr(t) for t in batch[0]['tokens'][0] if t))
plt.imshow(batch[0]['image'][0])

In [2]:
DATA_DIR = Path('../data/tfrecord')
TRAIN_DATA_PATH = DATA_DIR / 'training_data.tfrecord'
VALIDATION_DATA_PATH = DATA_DIR / 'validation_data.tfrecord'
TEST_DATA_PATH = DATA_DIR / 'testing_data.tfrecord'

training_data = data.make_dataset(str(TRAIN_DATA_PATH))
validation_data = data.make_dataset(str(VALIDATION_DATA_PATH))

In [3]:
importlib.reload(text_model)
importlib.reload(data)

config = text_model.Config(lstm_layers=1,
                           lstm_size=128,
                           embedding_size=128,
                           attention_num_heads=3,
                           attention_head_size=64,
                           feature_params={'vocab_size': 255},)
model = text_model.create_model(config)

model.fit(training_data,
          validation_data=validation_data,
          epochs=10,
          # metrics=['accuracy', 'val_acc']
          steps_per_epoch=111,
          validation_steps=11)

/home/derick/.conda/envs/tf-1.12/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10


  1/111 [..............................] - ETA: 1:20:40 - loss: 1.3340 - acc: 0.0469

  2/111 [..............................] - ETA: 40:26 - loss: 1.3313 - acc: 0.2969  

  3/111 [..............................] - ETA: 27:03 - loss: 1.3116 - acc: 0.3802

...

107/111 [===========================>..] - ETA: 2s - loss: 0.8619 - acc: 0.5618

108/111 [============================>.] - ETA: 1s - loss: 0.8620 - acc: 0.5618

109/111 [============================>.] - ETA: 1s - loss: 0.8615 - acc: 0.5619

110/111 [============================>.] - ETA: 0s - loss: 0.8619 - acc: 0.5616

111/111 [==============================] - 71s 643ms/step - loss: 0.8621 - acc: 0.5612 - val_loss: 0.8106 - val_acc: 0.5945


In [4]:
model.fit(training_data,
          validation_data=validation_data,
          epochs=10,
          # metrics=['accuracy', 'val_acc']
          steps_per_epoch=111,
          validation_steps=11)

Epoch 1/10


  1/111 [..............................] - ETA: 1:11:57 - loss: 0.9143 - acc: 0.5469

  2/111 [..............................] - ETA: 36:14 - loss: 0.8806 - acc: 0.5742  

  3/111 [..............................] - ETA: 24:18 - loss: 0.8484 - acc: 0.5885

  4/111 [>.............................] - ETA: 18:17 - loss: 0.8543 - acc: 0.5820

...

107/111 [===========================>..] - ETA: 2s - loss: 5.8087 - acc: 0.3714

108/111 [============================>.] - ETA: 1s - loss: 5.7734 - acc: 0.3733

109/111 [============================>.] - ETA: 1s - loss: 5.8186 - acc: 0.3713

110/111 [============================>.] - ETA: 0s - loss: 5.7773 - acc: 0.3730

111/111 [==============================] - 69s 622ms/step - loss: 5.8364 - acc: 0.3716 - val_loss: 7.3877 - val_acc: 0.1179


In [4]:
importlib.reload(text_model)
importlib.reload(data)

config = text_model.Config(lstm_layers=1,
                           lstm_size=256,
                           embedding_size=64,
                           attention_num_heads=3,
                           attention_head_size=64,
                           # embedding_regularization_coef=1e-4,
                           # dense_regularization_coef=1e-4,
                           feature_params={'vocab_size': 255},)
model = text_model.create_simple_model(config)

model.fit(training_data,
          validation_data=validation_data,
          epochs=30,
          # metrics=['accuracy', 'val_acc']
          steps_per_epoch=111,
          validation_steps=11)

/home/derick/.conda/envs/tf-1.12/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/30


  1/111 [..............................] - ETA: 1:14:46 - loss: 1.3007 - acc: 0.1562

  2/111 [..............................] - ETA: 37:37 - loss: 1.3087 - acc: 0.3398  

  3/111 [..............................] - ETA: 25:15 - loss: 1.2952 - acc: 0.4010

...

 46/111 [===========>..................] - ETA: 47s - loss: 1.3963 - acc: 0.3942

 47/111 [===========>..................] - ETA: 46s - loss: 1.3890 - acc: 0.3978

 48/111 [===========>..................] - ETA: 45s - loss: 1.3939 - acc: 0.3962

 49/111 [============>.................] - ETA: 44s - loss: 1.3910 - acc: 0.3997

KeyboardInterrupt: 

In [ ]:
np.softmax(model.predict(batch[0]))

In [9]:
tf.keras.backend.clear_session()